## Test Neuroevolución

In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler  

In [2]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import pickle
from copy import deepcopy
from sklearn import preprocessing

Importación del scaler
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

### Cargo los datos y retiro el Timestamp y el score

In [3]:
df = pd.read_csv("gameStates.csv", sep=',', header=None)
df.columns = ['timeStamp','Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6','ray1','ray2','ray3','ray4','ray5','ray6','ray7','ray8','ray9','ray10','ray11','ray12','ray13','ray14','ray15','ray16','ray17','ray18','ray19','ray20','ray21','ray22','ray23','ray24','ray25','ray26','ray27','score',"VKey","HKey","Shooting"]
df.drop(['timeStamp','score', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6'],axis=1,inplace=True)
df = df[1:9000]



#### Transforma el porblema multioputput en multilabel

In [4]:
def multioutput2multilabel(row):
    values = pd.Series([(row["VKey"] == "UpArrow"),(row["VKey"] == "DownArrow"),(row["HKey"] == "LeftArrow"),(row["HKey"] == "RightArrow"),row["Shooting"]])
    return pd.concat([row[:len(row)-3], values.astype(float)])


In [5]:
def concatenar_estados(dataframe):
        dfAnterior = dataframe.shift(1)
        dfAnterior.rename(columns=lambda x: str(x)+"-1", inplace=True)
        dfAnterior2 = dataframe.shift(2)
        dfAnterior2.rename(columns=lambda x: str(x)+"-2", inplace=True)
        dfAnterior3 = dataframe.shift(3)
        dfAnterior3.rename(columns=lambda x: str(x)+"-3", inplace=True)
        df = pd.concat([dfAnterior3,dfAnterior2,dfAnterior,dataframe],axis=1)
        df = df.fillna(999)
        return df

print(len(df.ix[1]))
df = df.apply(multioutput2multilabel,axis=1)
print(len(df.ix[1]))

37
39


In [6]:
df = concatenar_estados(df)
len(df.ix[1])

156

In [7]:
#Defino los elementos train_data y target_data
train = df.drop([0, 1, 2,3,4],axis=1).values
labels = df[[0,1,2,3,4]].values

### Pipeline

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

clf = MLPClassifier(solver='lbfgs', alpha=0.0001, learning_rate= 'constant',
                    random_state = 0, learning_rate_init = 1, 
                    hidden_layer_sizes=(10,), batch_size = 500,) 

estimators = [('stantandarize', StandardScaler()), ('clf', clf)]

pipe = Pipeline(estimators)



# entreno el pipe como si fuera un clasificador, 
# también podría hacer validación cruzada y todo como si fuese un clasificador normal
pipe.fit(train, labels)

pickle.dump(pipe, open("pipe.sav", 'wb'))

shapes = [coef.shape for coef in pipe.named_steps['clf'].coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in pipe.named_steps['clf'].coefs_]


from sklearn.model_selection import cross_val_score
print(cross_val_score(pipe, train, labels, cv=5).mean())

0.554394293126


### Neuroevolución

In [3]:
def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))

In [8]:
def MLPFitness(individual):
    coefs = gen2Coefs(np.array(individual),sizes,shapes)
    pipe.named_steps['clf'].coefs_ = coefs 
    pickle.dump(pipe, open("temp.sav", 'wb'))
    os.system("AI.exe 10 60 ..\\temp.sav")
    scores = pd.read_csv("myData.csv", sep=',', header=None)
    os.remove("myData.csv")
    f = np.mean(scores.values)/np.std(scores.values)
    return (f,)



MLPFitness(coefs2gen(pipe.named_steps['clf'].coefs_,sizes,shapes))

(1.7936862216329246,)

In [5]:
def initPopulation(pop, ind_random, ind_guess,n_guess=1,n=10):
    
    pop = []
    n_random = n - n_guess
    for i in range(n_random):
        pop.append(ind_random())
        
    for i in range(n_guess):
        pop.append(ind_guess())
    
    return pop

In [ ]:


N = 2

pipe = pickle.load(open("elMejor.sav", 'rb')) #clasificador del que partimos
shapes = [coef.shape for coef in pipe.named_steps['clf'].coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in pipe.named_steps['clf'].coefs_]
pesosIniciales = coefs2gen(pipe.named_steps['clf'].coefs_,sizes,shapes).copy()


creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximiza
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=sum(sizes))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("individual_guess", lambda :creator.Individual(pesosIniciales))
toolbox.register("population_mix",initPopulation,list,toolbox.individual, toolbox.individual_guess)       


toolbox.register("evaluate", MLPFitness)
toolbox.register("mate", tools.cxOnePoint)

toolbox.register("mutate", tools.mutGaussian, mu=0.5, sigma=0.5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=10)

random.seed(64)
  
population = toolbox.population_mix(n=10, n_guess=N)  
    
hof = tools.HallOfFame(1, similar=np.array_equal)
    
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
    
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hof)
#population, logbook = algorithms.eaMuPlusLambda(population, toolbox,mu = 4, lambda_= 10, cxpb=0.5, mutpb=0.05, ngen=10, stats=stats, halloffame=hof)
print(hof) # hall of the fame contiene el mejor individuo vivo en cada generacion
mejorInd = hof[0] # esta ordenado de manera que el primer elemento es el mejor de siempre

MLPFitness(mejorInd)
#send_message("Done!", 4191538)
#send_message(MLPFitness(mejorInd), 4191538)
#Guarda el mejor
coefs = gen2Coefs(np.array(mejorInd),sizes,shapes)
pipe.named_steps['clf'].coefs_ = coefs 
pickle.dump(pipe, open("pipe.sav", 'wb'))

0  	5     	-180.767	80.8397	-287.219	-95.0069
1  	2     	-133.332	52.4052	-227.667	-95.0069
2  	3     	-158.725	53.4109	-213.969	-95.0069

In [7]:
coefs = gen2Coefs(np.array(mejorInd),sizes,shapes)
pipe.named_steps['clf'].coefs_ = coefs 
pickle.dump(pipe, open("elMejor.sav", 'wb'))

In [34]:
from sklearn.model_selection import cross_val_score
pipe = pickle.load(open("elMejor.sav", 'rb'))
print(cross_val_score(pipe, train, labels, cv=5).mean())

mejorInd = hof[0]
MLPFitness(mejorInd)

0.554394293126


(2016.8,)

### Calibrado de parámetros

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import randint as sp_randint


# especifica los parámetros y su distribución
## tamaño capa oculta
## Learning rate intit


param_dist = {"clf__hidden_layer_sizes":[(2,),(5,),(10,),(50,),(100,)],
              "clf__solver":['lbfgs'],
              "clf__batch_size":[100,200,500],
              "clf__learning_rate_init":[0.001,0.0001,0.0005,0.1,1],
              "clf__learning_rate":["constant", "invscaling", "adaptive"],
              "clf__alpha":[0.001,0.0001,0.0005,0.1,1,5,10],
              "clf__random_state":[0,1,10]}

# ejecuta búsqueda aleatoria
num_iteraciones = 200 ## aumentar numero
random_search = RandomizedSearchCV(pipe, param_distributions=param_dist,
                                   n_iter=num_iteraciones)

#clf.get_params().keys()
random_search.fit(train, labels)
#send_message("Done!", 4191538)
##send_message(random_search.best_params_,4191538)

NameError: name 'send_message' is not defined

In [13]:
print("Los mejores parámetros encontrados son:")
print()
print(random_search.best_params_)

Los mejores parámetros encontrados son:

{'clf__solver': 'lbfgs', 'clf__random_state': 0, 'clf__learning_rate_init': 1, 'clf__learning_rate': 'invscaling', 'clf__hidden_layer_sizes': (5,), 'clf__batch_size': 500, 'clf__alpha': 0.0001}


### Telegram bot

In [ ]:
import json 
import requests

TOKEN = "371099199:AAFy_hyhXjc4Ew8pGS1engI6RdJ3YDT0VSU"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)

In [ ]:
def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content

def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js

def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js

def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (textchat_id)
    print (get_last_chat_id_and_text(updates))

def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)

In [ ]:
text, chat = get_last_chat_id_and_text(get_updates())
#send_message(text, chat)
print(chat)
send_message("Hola!", 4191538)

In [85]:
random.random


<function Random.random>

In [9]:
pipe = pickle.load(open("pipe.sav", 'rb'))
print(pipe)

Pipeline(steps=[('stantandarize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', MLPClassifier(activation='relu', alpha=0.0001, batch_size=500, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_...      solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))])
